In [38]:
!pip install tensorflow==2.19.0

In [75]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
import os
import random
import pathlib

In [76]:
image_dir = "C:/Users/HP/Desktop/ML_DS/CNN/data/oxford-iiit-pet/images/images"
label_file = "C:/Users/HP/Desktop/ML_DS/CNN/data/oxford-iiit-pet/annotations/annotations/list.txt"

with open(label_file, 'r') as f:
    lines = f.readlines()[6:] 

data = []
for line in lines:
    parts = line.strip().split()
    if len(parts) >= 2:
        filename = parts[0] + ".jpg"
        label = "_".join(parts[0].split("_")[:-1])
        full_path = os.path.join(image_dir, filename)
        if os.path.exists(full_path):
            data.append((full_path, label))


In [77]:
label_names = sorted(set(label for _, label in data))
label2idx = {label: idx for idx, label in enumerate(label_names)}
data = [(img, label2idx[label]) for img, label in data]

train_val_data, test_data = train_test_split(data, test_size=0.15, stratify=[label for _, label in data], random_state=42)
train_data, val_data = train_test_split(train_val_data, test_size=0.15, stratify=[label for _, label in train_val_data], random_state=42)

def build_dataset(data_list):
    paths, labels = zip(*data_list)
    ds = tf.data.Dataset.from_tensor_slices((list(paths), list(labels)))

    def load_and_preprocess(path, label):
        image = tf.io.read_file(path)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (224, 224))
        image = preprocess_input(image)
        return image, label

    return ds.map(load_and_preprocess).batch(32).prefetch(tf.data.AUTOTUNE)

train_ds = build_dataset(train_data)
val_ds = build_dataset(val_data)
test_ds = build_dataset(test_data)



In [78]:
num_classes = len(label2idx)
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [79]:
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_6         │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_3     │ (None, 224, 224,  │          7 │ rescaling_6[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_7         │ (None, 224, 224,  │          0 │ normalization_3[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_7[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,096,968 (15.63 MB)

 Trainable params: 47,397 (185.14 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [80]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [81]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
os.makedirs("checkpoints", exist_ok=True)
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint("checkpoints/best_model.h5", save_best_only=True, monitor='val_loss')
]

In [82]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=20,
          callbacks=callbacks)

Epoch 1/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5593 - loss: 1.9884

166/166 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.5605 - loss: 1.9835 - val_accuracy: 0.9029 - val_loss: 0.4395
Epoch 2/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9073 - loss: 0.3853

166/166 ━━━━━━━━━━━━━━━━━━━━ 221s 1s/step - accuracy: 0.9073 - loss: 0.3851 - val_accuracy: 0.9125 - val_loss: 0.3164
Epoch 3/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 892ms/step - accuracy: 0.9407 - loss: 0.2550

166/166 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - accuracy: 0.9407 - loss: 0.2550 - val_accuracy: 0.9221 - val_loss: 0.2763
Epoch 4/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9496 - loss: 0.2037

166/166 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - accuracy: 0.9496 - loss: 0.2037 - val_accuracy: 0.9264 - val_loss: 0.2562
Epoch 5/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 894ms/step - accuracy: 0.9594 - loss: 0.1656

...

166/166 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.9744 - loss: 0.1103 - val_accuracy: 0.9338 - val_loss: 0.2331
Epoch 9/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9733 - loss: 0.1027

166/166 ━━━━━━━━━━━━━━━━━━━━ 239s 1s/step - accuracy: 0.9733 - loss: 0.1027 - val_accuracy: 0.9338 - val_loss: 0.2322
Epoch 10/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 890ms/step - accuracy: 0.9782 - loss: 0.0914

166/166 ━━━━━━━━━━━━━━━━━━━━ 169s 1s/step - accuracy: 0.9781 - loss: 0.0914 - val_accuracy: 0.9338 - val_loss: 0.2290
Epoch 11/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 210s 1s/step - accuracy: 0.9795 - loss: 0.0881 - val_accuracy: 0.9338 - val_loss: 0.2315
Epoch 12/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 682ms/step - accuracy: 0.9833 - loss: 0.0757

166/166 ━━━━━━━━━━━━━━━━━━━━ 141s 849ms/step - accuracy: 0.9833 - loss: 0.0757 - val_accuracy: 0.9274 - val_loss: 0.2289
Epoch 13/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.9849 - loss: 0.0706 - val_accuracy: 0.9328 - val_loss: 0.2291
Epoch 14/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9884 - loss: 0.0646

166/166 ━━━━━━━━━━━━━━━━━━━━ 273s 2s/step - accuracy: 0.9884 - loss: 0.0646 - val_accuracy: 0.9338 - val_loss: 0.2252
Epoch 15/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 147s 878ms/step - accuracy: 0.9867 - loss: 0.0635 - val_accuracy: 0.9328 - val_loss: 0.2271
Epoch 16/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 118s 710ms/step - accuracy: 0.9872 - loss: 0.0579 - val_accuracy: 0.9328 - val_loss: 0.2253
Epoch 17/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 121s 732ms/step - accuracy: 0.9925 - loss: 0.0509 - val_accuracy: 0.9338 - val_loss: 0.2337
